<img style="float: center;" src='./assets/jWST-SS-1300x300_Banner2.jpg' width="1000px"/> 

In [6]:
from matplotlib import pyplot as plt
import numpy as np
from astropy.table import Table
import os,shutil,glob,sys,traceback
from astropy.cosmology import LambdaCDM
import pdb
import pickle
from astropy.io import fits
from astroquery.mast import Observations
import jhat
from jhat import jwst_photclass,hst_photclass,st_wcs_align
import space_phot
import astropy
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time
from jwst.pipeline import Image3Pipeline
import pprint
from astroquery.mast import Observations
import subprocess
import sewpy
from astropy.wcs import WCS

In [7]:
JADES_PID = '1180'

# Get supernova positions
# https://www.wis-tns.org/object/2023adta
SN2023adta = SkyCoord(53.135283, -27.814517, unit=(u.deg, u.deg), frame='icrs')
rad=3.0/60.0 # 3 arcmin in deg

obs_table = Observations.query_criteria(
                                        coordinates=f'{SN2023adta.ra.degree} {SN2023adta.dec.degree}',
                                        radius=f"{rad} deg",
                                        instrument_name=['NIRCAM','NIRCAM/IMAGE'],
                                        proposal_id = [JADES_PID],
                                        calib_level=["3"],
                                        filters=['F200W','F356W']
                                        )
obs_table

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID,objID1,distance
str7,str4,str7,str12,str4,str5,str8,str29,str33,str36,float64,float64,str5,str21,int64,float64,float64,float64,float64,float64,str62,float64,str4,str3,int64,str115,str62,str63,str6,bool,float64,str9,str9,str9,float64
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F356W,INFRARED,MEDS0001-replan,Unidentified; High Latitude Field,jw01180-o219_t219_nircam_clear-f356w,53.16163458333334,-27.822508333333303,image,"Eisenstein, Daniel J.",3,60263.59145759259,60263.66229165509,5669.0160000000005,3140.0,3980.0,NIRCam-NIRSpec galaxy assembly survey - GOODS-S - part #1athen,60631.64543972,1180,GTO,--,POLYGON 53.112599997 -27.846796753 53.109456757 -27.806370889 53.209371694 -27.800258675 53.212551967 -27.840682262,mast:JWST/product/jw01180-o219_t219_nircam_clear-f356w_i2d.jpg,mast:JWST/product/jw01180-o219_t219_nircam_clear-f356w_i2d.fits,PUBLIC,False,nan,192112125,810208411,810208411,0.0
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F356W,INFRARED,POINTINGFOUR-A,Unidentified; High Latitude Field,jw01180-o018_t014_nircam_clear-f356w,53.152095833333334,-27.769305555555547,image,"Eisenstein, Daniel J.",3,59854.60018243056,59854.75178936343,12368.763000000003,3140.0,3980.0,NIRCam-NIRSpec galaxy assembly survey - GOODS-S - part #1athen,60100.7916666,1180,GTO,--,POLYGON 53.147622255 -27.815774569 53.110412503 -27.797948358 53.156454715 -27.722717262 53.193646284 -27.740531151,mast:JWST/product/jw01180-o018_t014_nircam_clear-f356w_i2d.jpg,mast:JWST/product/jw01180-o018_t014_nircam_clear-f356w_i2d.fits,PUBLIC,False,nan,224093147,811206561,811206561,14.726676099577592
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F200W,INFRARED,GS-MEDIUM-HST,--,jw01180-o028_t028_nircam_clear-f200w,53.15372125,-27.780369444444432,image,"Eisenstein, Daniel J.",3,59860.96615611111,59861.15206190972,5669.0160000000005,1755.0,2226.0,NIRCam-NIRSpec galaxy assembly survey - GOODS-S - part #1athen,60226.34187503,1180,GTO,--,POLYGON 53.08388357 -27.904560956 53.048151945 -27.880367717 53.107414009 -27.811954152 53.143136306 -27.836132126,mast:JWST/product/jw01180-o028_t028_nircam_clear-f200w_i2d.jpg,mast:JWST/product/jw01180-o028_t028_nircam_clear-f200w_i2d.fits,PUBLIC,False,nan,224908641,810475284,810475284,46.60638185210617
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F200W,INFRARED,MEDS0002a,Unidentified; High Latitude Field,jw01180-o220_t220_nircam_clear-f200w,53.20424,-27.795958333333317,image,"Eisenstein, Daniel J.",3,60310.14977819444,60310.213904386575,5024.808,1755.0,2226.0,NIRCam-NIRSpec galaxy assembly survey - GOODS-S - part #1athen,60676.52178232,1180,GTO,--,POLYGON 53.1300604 -27.765260805 53.161863735 -27.734509483 53.236315105 -27.794754521 53.204515209 -27.825522884,mast:JWST/product/jw01180-o220_t220_nircam_clear-f200w_i2d.jpg,mast:JWST/product/jw01180-o220_t220_nircam_clear-f200w_i2d.fits,PUBLIC,False,nan,199975191,809954108,809954108,119.56101134798543
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F200W,INFRARED,GS-MEDIUM-HST,--,jw01180-o030_t028_nircam_clear-f200w,53.15372125,-27.780369444444432,image,"Eisenstein, Daniel J.",3,59862.76432083333,59862.80147789352,2834.508,1755.0,2226.0,NIRCam-NIRSpec galaxy assembly survey - GOODS-S - part #1athen,60228.68001146,1180,GTO,--,POLYGON 53.101777837 -27.938477162 53.067470688 -27.915238448 53.126558376 -27.847098384 53.16085661 -27.870322472,mast:JWST/product/jw01180-o030_t028_nircam_clear-f200w_i2d.jpg,mast:JWST/product/jw01180-o030_t028_nircam_clear-f200w_i2d.fits,PUBLIC,False,nan,97276909,810418803,810418803,120.53681652943298
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F200W,INFRARED,GS-MEDIUM-HST,--,jw01180-o026_t028_nircam_clear-f200w,53.15372125,-27.780369444444432,image,"Eisenstein

In [8]:
prod = Observations.get_unique_product_list(obs_table)
prod

INFO: 311499 of 336960 products were duplicates. Only returning 25461 unique product(s). [astroquery.mast.utils]
INFO: To return all products, use `Observations.get_product_list` [astroquery.mast.observations]


obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str9,str4,str5,str36,str64,str1,str69,str9,str28,str11,str1,str7,str6,str4,str51,int64,str9,str6,int64,str5
223810605,JWST,image,jw01180007001_10101_00009_nrcb4,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00011_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00011_asn.json,1803,224007745,PUBLIC,2,F200W
223773483,JWST,image,jw01180007001_10101_00009_nrcb3,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00012_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00012_asn.json,1803,224007745,PUBLIC,2,F200W
223852702,JWST,image,jw01180007001_10101_00009_nrcb2,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00013_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00013_asn.json,1803,224007745,PUBLIC,2,F200W
95382691,JWST,image,jw01180007001_10101_00009_nrcb1,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00014_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00014_asn.json,1803,224007745,PUBLIC,2,F200W
223929256,JWST,image,jw01180007001_10101_00009_nrca4,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00016_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00016_asn.json,1803,224007745,PUBLIC,2,F200W
95382517,JWST,image,jw01180007001_10101_00009_nrca3,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00017_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00017_asn.json,1803,224007745,PUBLIC,2,F200W
223784156,JWST,image,jw01180007001_10101_00009_nrca2,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00018_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00018_asn.json,1803,224007745,PUBLIC,2,F200W
223919306,JWST,image,jw01180007001_10101_00009_nrca1,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00019_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00019_asn.json,1803,224007745,PUBLIC,2,F200W
223882285,JWST,image,jw01180007001_10101_00008_nrcb4,source/target (L3) : association generator,S,mast:JWST/product/jw01180-o007_20250525t210127_image2_00021_asn.json,INFO,--,ASN,--,CALJWST,1.18.0,1180,jw01180-o007_20250525t210127_image2_00021_asn.json,1803,224007745,PUBLIC,2,F200W


In [9]:
products = Observations.filter_products(prod,
                                        productSubGroupDescription=["CAL"],
                                        extension="fits")
products

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str9,str4,str5,str36,str64,str1,str69,str9,str28,str11,str1,str7,str6,str4,str51,int64,str9,str6,int64,str5
95382616,JWST,image,jw01180007001_06101_00001_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00001_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00001_nrcalong_cal.fits,117573120,224007792,PUBLIC,2,F356W
223922492,JWST,image,jw01180007001_06101_00001_nrcblong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00001_nrcblong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00001_nrcblong_cal.fits,117573120,224007792,PUBLIC,2,F356W
95382563,JWST,image,jw01180007001_06101_00002_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00002_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00002_nrcalong_cal.fits,117573120,224007792,PUBLIC,2,F356W
95383048,JWST,image,jw01180007001_06101_00002_nrcblong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00002_nrcblong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00002_nrcblong_cal.fits,117573120,224007792,PUBLIC,2,F356W
223881812,JWST,image,jw01180007001_06101_00003_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00003_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00003_nrcalong_cal.fits,117573120,224007792,PUBLIC,2,F356W
95382947,JWST,image,jw01180007001_06101_00003_nrcblong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00003_nrcblong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00003_nrcblong_cal.fits,117573120,224007792,PUBLIC,2,F356W
223861921,JWST,image,jw01180007001_06101_00004_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00004_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00004_nrcalong_cal.fits,117573120,224007792,PUBLIC,2,F356W
95383023,JWST,image,jw01180007001_06101_00004_nrcblong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00004_nrcblong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00004_nrcblong_cal.fits,117573120,224007792,PUBLIC,2,F356W
95382620,JWST,image,jw01180007001_06101_00005_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw01180007001_06101_00005_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.18.0,1180,jw01180007001_06101_00005_nrcalong_cal.fits,117573120,224007792,PUBLIC,2,F356W


In [10]:
epoch_1 = ["jw01180015001_10101_00006_nrcb2",
"jw01180015001_10101_00003_nrcb2",
"jw01180015001_10101_00008_nrcb2",
"jw01180015001_10101_00009_nrcb2",
"jw01180015001_10101_00007_nrcb2",
"jw01180015001_10101_00001_nrcb2",
"jw01180015001_10101_00004_nrcb2",
"jw01180015001_10101_00005_nrcb2",
"jw01180015001_10101_00002_nrcb2",
"jw01180015001_06101_00007_nrcblong",
"jw01180015001_06101_00005_nrcblong",
"jw01180015001_06101_00001_nrcblong",
"jw01180015001_06101_00002_nrcblong",
"jw01180015001_06101_00006_nrcblong",
"jw01180015001_06101_00004_nrcblong",
"jw01180015001_06101_00009_nrcblong",
"jw01180015001_06101_00003_nrcblong",
"jw01180015001_06101_00008_nrcblong"]

epoch_2 =[
    "jw01180013001_10101_00009_nrcb2",
    "jw01180013001_10101_00002_nrcb2",
    "jw01180013001_10101_00007_nrcb2",
    "jw01180013001_10101_00006_nrcb2",
    "jw01180013001_10101_00003_nrcb2",
    "jw01180013001_10101_00001_nrcb2",
    "jw01180013001_10101_00008_nrcb2",
    "jw01180013001_10101_00004_nrcb2",
    "jw01180013001_10101_00005_nrcb2",
    "jw01180013001_06101_00005_nrcblong",
    "jw01180013001_06101_00004_nrcblong",
    "jw01180013001_06101_00006_nrcblong",
    "jw01180013001_06101_00002_nrcblong",
    "jw01180013001_06101_00008_nrcblong",
    "jw01180013001_06101_00009_nrcblong",
    "jw01180013001_06101_00001_nrcblong",
    "jw01180013001_06101_00007_nrcblong",
    "jw01180013001_06101_00003_nrcblong"
]



products_epoch1 = Observations.filter_products(prod,
                                        productSubGroupDescription=["CAL"],
                                        extension="fits",
                                        obs_id=epoch_1)

products_epoch2 = Observations.filter_products(prod,
                                        productSubGroupDescription=["CAL"],
                                        extension="fits",
                                        obs_id=epoch_2)

In [12]:
os.makedirs("./JADES_data/epoch1", exist_ok=True)
os.makedirs("./JADES_data/epoch2", exist_ok=True)

In [13]:
manifest1 = Observations.download_products(products_epoch1, download_dir="./JADES_data/epoch1", flat=True)


In [15]:
manifest2 = Observations.download_products(products_epoch2, download_dir="./JADES_data/epoch2", flat=True)